In [1]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt

# A.1 No exponential magic

Quantum computers use qubits that can be in a superposition of quantum states 0 and 1. When multiple qubits are in superposition, they form a tensor product, which is a superposition over every combination of the qubits' quantum states. This creates an exponential number of terms in the superposition, but when a measurement is taken, only one term is observed, giving a random snapshot of the system. The challenge is to design algorithms that shuffle the exponential collection of terms so that the measurement has a high probability of giving the desired result. Simply having a superposition does not automatically result in an exponential speedup compared to classical computing, as the outcome is still probabilistic and subject to randomness.

## Codercise A.1.1.

 Fill in the following code to create the uniform superposition over $n$ qubits. It will plot the probability of observing different outcomes. Does putting the circuit in a uniform superposition help us break the lock?

Note. The pass is a placeholder you will replace with your code.

The goal of the task is to create a uniform superposition over all possible -bit strings, and then measure the qubits to see if the lock's combination is obtained. To do this, the Hadamard gate is applied to each of the qubits. Then, the final state is measured and the probabilities of observing each outcome are calculated. The question asks whether putting the circuit in a uniform superposition helps break the lock, which is answered by examining the resulting probabilities.

In [2]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)


@qml.qnode(dev)
def naive_circuit():
    """Create a uniform superposition and return the probabilities.

    Returns:
        array[float]: Probabilities for observing different outcomes.
    """
    for wire in range(n_bits):
        qml.Hadamard(wires=wire)

    return qml.probs(wires=range(n_bits))

The code defines a quantum function `naive_circuit` that runs on a quantum device dev with `n_bits = 4` wires. The function creates a uniform superposition on each wire by applying a Hadamard gate to each wire. Finally, the function returns the probabilities for observing different outcomes by calling the `qml.probs` function on the range of all wires.

# A.2 The magic 8-ball

In quantum algorithms, an oracle is a black box that performs some function and provides a result to the algorithm, without revealing the inner workings of the function. The function is modeled as a unitary matrix applied to computational basis states. The quantum version of breaking a lock can be thought of as asking the oracle if a combination works, and the oracle provides a "yes" or "no" answer by putting the answer into a phase. This is done by encoding the answer as a unitary operator that acts on basis states. The oracle can be written in terms of the identity matrix and the projector, and it can be shown that the oracle is unitary.

## Codercise A.2.1.

Write a function which returns the oracle in matrix form for a given secret combination.

The simplest way to do this is to create the identity matrix $\mathbb{I}$ and then flip the sign of the amplitude corresponding to the correct combination using the projector:

$$
U_f = \mathbb{I} - 2 \ket{s} \bra{s}
$$

This represents the oracle operation that changes the sign of the amplitude corresponding to the correct combination. In quantum algorithms, oracles are used to determine if a guess is correct or not, and the oracle's action is modeled as a unitary matrix applied to computational basis states.

In [3]:
def oracle_matrix(combo):
    """Return the oracle matrix for a secret combination.

    Args:
        combo (list[int]): A list of bits representing a secret combination.

    Returns:
        array[float]: The matrix representation of the oracle.
    """
    index = np.ravel_multi_index(combo, [2] * len(combo))  # Index of solution
    my_array = np.identity(2 ** len(combo))  # Create the identity matrix

    my_array[index, index] = -1

    return my_array

This function takes in a list `combo` of bits representing a secret combination and returns the matrix representation of the oracle for that combination. The function first calculates the index of the solution. It then creates the identity matrix and changes the sign of the element in the index corresponding to the solution. Finally, it returns the modified identity matrix as the oracle matrix.

## Codercise A.2.2

Write a circuit which applies the oracle to the uniform superposition. The oracle matrix function from the previous exercise is available for you as `oracle_matrix`. The supplied code will plot the resulting probability distribution. Has applying the oracle helped us break the lock?

Tip. Use PennyLane's `QubitUnitary()` operation.

In [4]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)


@qml.qnode(dev)
def oracle_circuit(combo):
    """
    Define the oracle circuit for a given combination `combo`

    Args:
    ------

        `combo` (`int` or `str`): The secret combination to be used in the oracle circuit.

    Returns:
    --------

        `array`: The probabilities of each bit being measured in the '0' state or the '1' state.
    """
    # Uniform superposition
    for wire in range(n_bits):
        qml.Hadamard(wires=wire)

    # Apply U_f
    qml.QubitUnitary(oracle_matrix(combo), wires=list(range(n_bits)))

    # Return probabilities
    return qml.probs(wires=range(n_bits))

This code implements the oracle circuit. It starts by preparing a uniform superposition of all computational basis states on `n_bits` qubits using the `Hadamard` gate. Then it applies the oracle operation defined by the `oracle_matrix` function to each qubit. Finally, it returns the probabilities of measuring each basis state by calling the `probs` function. The `oracle_matrix` function takes a secret combination as input and returns the oracle matrix representation for this combination.

# A.3 Pair programming

In pair programming, the oracle is applied to a pair of candidate solutions to determine if the secret combination is present. The algorithm works by dividing the full set of strings into pairs based on the first $n-1$ bits, labeled by $\ket{\tilde{x}}$. The linear combination of the two solutions in the pair is then applied to the oracle, which results in a phase change in one of the solutions. The last qubit is then measured to determine which solution has undergone a phase change. If the solution has undergone a phase change, then the correct combination is either the first or the second solution in the pair.

The average number of queries required to find a solution when testing in pairs is $2^{n-1}$. For an n-bit solution, if the position of the combination is random and uniform in the list, the average number of pairs required to test is $\frac{2^{n-1} + 1}{2}$.

## Codercise A.3.1. 

Implement this circuit and return the probabilities on the last qubit. The function `oracle_matrix` is defined for you. 

The strategy for finding the solution using the oracle is to iterate through all n-bit strings in pairs, labelled by an n-bit string, and apply the oracle to the superposition. If the solution is present, there will be a relative phase change that can be observed by applying the Hadamard operator to the last qubit. The final state of the last qubit will be in the state if the solution is present, and if it is not. The average number of pairs required to find the solution is $\frac{2^{n-1} + 1}{2}$.

In [5]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)


@qml.qnode(dev)
def pair_circuit(x_tilde, combo):
    """Test a pair labelled by `x_tilde` for the presence of a solution.

    Args:
    ------

        `x_tilde` (`list[int]`): An `(n_bits - 1)`-string labelling the pair to test.
        `combo` (`list[int]`): A secret combination of `n_bits` 0s and 1s.

    Returns:
    --------

        `array[float]`: Probabilities on the last qubit.
    """
    for i in range(n_bits - 1):  # Initialize x_tilde part of state
        if x_tilde[i] == 1:
            qml.PauliX(wires=i)

    # Apply the circuit
    qml.Hadamard(wires=n_bits - 1)
    qml.QubitUnitary(oracle_matrix(combo), wires=list(range(n_bits)))
    qml.Hadamard(wires=n_bits - 1)

    return qml.probs(wires=n_bits - 1)

This code defines a quantum circuit called `pair_circuit` which is intended to test a pair of binary strings for the presence of a solution. The `pair_circuit` function takes two inputs, `x_tilde` and `combo`. The `x_tilde` argument is a list of `n_bits - 1` 0s and 1s that labels the pair being tested, and `combo` is a secret combination of `n_bits` 0s and 1s.

The circuit starts by initializing the `x_tilde` part of the quantum state by applying a Pauli X gate to the qubits corresponding to 1s in the `x_tilde` string. The circuit then applies a `Hadamard` gate to the last qubit, followed by the oracle circuit, and then another `Hadamard` gate to the last qubit. The final result is the probabilities of observing 0 or 1 on the last qubit, which are returned by the `qml.probs` function. The device the circuit runs on is defined as the default qubit device with `n_bits` wires.

## Codercise A.3.2.

Complete the function below to see how many attempts it takes to break the lock using our quantum circuit. You should find an improvement over the brute force approach, which takes around 9 guesses (on average) for 4 qubits. Note that `pair_circuit` is available.

Tip. Use `np.isclose(a, b)` to test the probabilities coming from `pair_circuit(x_tilde, combo)`.

In [6]:
import random


def secret_combo(n_bits):
    """Generates a random boolean number of length `n_bits`.

    Args:
    ------
        `n_bits` (int): Length of the boolean number to generate.

    Returns:
    -------
        list[int]: A list of 0s and 1s of length `n_bits`.

    Example:
    --------
    >>> secret_combo(3)
    [1, 0, 1]
    """
    return [random.choice([0, 1]) for i in range(n_bits)]

The `secret_combo` function generates a random boolean number of length `n_bits` by using a for loop to repeatedly choose a random 0 or 1 and append it to a list. The function takes as input an integer `n_bits` which specifies the length of the boolean number to generate. The output of the function is a list of `n_bits` elements, each of which is either 0 or 1, representing the random boolean number. The function uses the `random.choice` function from the random module to choose either 0 or 1 randomly in each iteration of the for loop.

In [7]:
def pair_lock_picker(trials):
    """Create a `combo`, run `pair_circuit` until it succeeds, and tally success rate.

    Args:
    ------

        `trials` (`int`): Number of times to test the lock picker.

    Returns:
    ---------

        `float`: The average number of times the lock picker uses `pair_circuit`.
    """
    x_tilde_strs = [np.binary_repr(n, n_bits - 1) for n in range(2 ** (n_bits - 1))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs]

    test_numbers = []

    for trial in range(trials):
        combo = secret_combo(n_bits)  # Random list of bits
        counter = 0
        for x_tilde in x_tildes:
            counter += 1

            if np.isclose(pair_circuit(x_tilde, combo)[1], 1):
                break

        test_numbers.append(counter)
    return sum(test_numbers) / trials


trials = 500
output = pair_lock_picker(trials)

print(f"For {n_bits} bits, it takes", output, "pair tests on average.")

For 4 bits, it takes 4.378 pair tests on average.


The function `pair_lock_picker` implements the pair testing algorithm for the lock picking problem. It creates a secret combination combo of bits, and then repeatedly tests pairs of bits using the `pair_circuit` function until it succeeds in finding the solution. The average number of pair tests required over trials trials is returned as the output of the function.

# A.4 Making and breaking promises

The author is discussing the importance of global information and promises in the design of algorithms. They use the example of a lock-picking algorithm to illustrate their point. The algorithm works by testing pairs of combinations to see if they open the lock. If the number of solutions (i.e. combinations that open the lock) is odd, the algorithm is guaranteed to eventually find a solution. However, if the number of solutions is even, the algorithm might get unlucky and miss the solutions because they come in pairs. In that case, the algorithm can still determine the parity of the solution set by counting the number of phase changes it observes. This gives the algorithm a way to determine global information about the function, in this case, the parity of the solution set. The author notes that global assumptions about the function, such as the parity of the solution set, can greatly impact the performance of the algorithm. These global assumptions are called "promises" and are an important aspect of quantum algorithms.

## Codercise A.4.1. 

<center>
<img src='./img/a.4.1.png' style='width:40%; height:40%;'>
</center>

Implement the circuit above, but now for `how_many` solutions instead of one. You will first need to implement the multi-solution version of the oracle matrix, `multisol_oracle_matrix(combos)`, which takes a list of bit strings (representing different solutions) and returns the associated oracle in matrix form.

The idea of "making and breaking promises" refers to the role that global information and assumptions play in the design of quantum algorithms. If there is a specific guarantee (a "promise") about the number of solutions to a problem, this can inform the design of the algorithm and help ensure that a solution can be found. However, if such a guarantee is not known in advance, testing in pairs can still reveal global information about the function, such as the parity of the number of solutions. In either case, the promise or the result of pair testing can have a significant impact on the performance and effectiveness of the algorithm.

In [8]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)


def multisol_oracle_matrix(combos):
    """Return the oracle matrix for a set of solutions.

    Args:
    -----

        `combos` (`list[list[int]]`): A list of secret bit strings.

    Returns:
    --------

        `array[float]`: The matrix representation of the oracle.
    """
    indices = [np.ravel_multi_index(combo, [2] * len(combo)) for combo in combos]
    my_array = np.identity(2**n_bits)
    for i in range(len(combos)):
        my_array[indices[i], indices[i]] = -1
    return my_array


@qml.qnode(dev)
def multisol_pair_circuit(x_tilde, combos):
    """Implements the circuit for testing a pair of combinations labelled by x_tilde.

    Args:
        x_tilde (list[int]): An (n_bits - 1)-bit string labelling the pair to test.
        combos (list[list[int]]): A list of secret bit strings.

    Returns:
        array[float]: Probabilities on the last qubit.
    """
    for i in range(n_bits - 1):  # Initialize x_tilde part of state
        if x_tilde[i] == 1:
            qml.PauliX(wires=i)

    # Apply circuit
    qml.Hadamard(wires=n_bits - 1)
    qml.QubitUnitary(multisol_oracle_matrix(combos), wires=list(range(n_bits)))
    qml.Hadamard(wires=n_bits - 1)

    return qml.probs(wires=n_bits - 1)

This code defines a quantum circuit and its supporting functions to test a pair of combinations in the case of multiple solutions. The circuit is meant to work for a quantum device with `n_bits` qubits, where `n_bits` is specified as 4 in this case. The device is initialized using the `qml.device` function and the circuit is defined using the `qml.qnode` decorator.

The `multisol_oracle_matrix` function takes as input a list of secret bit strings, called `combos`, and returns the matrix representation of the oracle for those combinations. This matrix is constructed by starting with the identity matrix of size `2**n_bits` and flipping the sign of the diagonal elements corresponding to the indices of the `combos` bit strings.

The `multisol_pair_circuit` function implements the circuit to test a pair of combinations. It takes as input a `(n_bits - 1)`-bit string `x_tilde` that labels the pair to test, and the combos list of secret bit strings. The circuit initializes the `x_tilde` part of the state, applies the Hadamard gate to the last qubit, applies the oracle matrix, and then applies the Hadamard gate to the last qubit again. The function returns the probabilities on the last qubit.

## Codercise A.4.2.

Use `multisol_pair_circuit(x_tilde, combos)` to create a routine for checking the parity of a lock.

In [9]:
def parity_checker(combos):
    """Use `multisol_pair_circuit` to determine the parity of a solution set.

    Args:
    ------

        `combos` (`list[list[int]]`): A list of secret combinations.

    Returns:
    --------

        `int`: The parity of the solution set.
    """
    parity = 0
    x_tilde_strs = [np.binary_repr(n, n_bits - 1) for n in range(2 ** (n_bits - 1))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs]

    counter = 0
    # IMPLEMENT PARITY COUNTING ALGORITHM
    for x_tilde in x_tildes:
        if np.isclose(multisol_pair_circuit(x_tilde, combos)[1], 1):
            counter += 1

    parity = counter % 2

    return parity

This code implements a quantum algorithm to determine the parity of a set of solutions. It does so by using the quantum circuit defined in `multisol_pair_circuit`, which takes as input a binary string of length `n_bits - 1` and a list of solutions. The function `multisol_pair_circuit` returns the probabilities of the last qubit being in the states $\ket{0}$ and $\ket{1}$.

The function `parity_checker` takes as input a list of solutions and calculates the parity of the set by using `multisol_pair_circuit` for each possible `n_bits - 1` bit string, referred to as `x_tilde`, and counting the number of times that the last qubit is in the $\ket{1}$ state. The final parity is calculated as the count of $\ket{1}$ states modulo 2.

# A.5 Hadamard transform

This text is discussing the Hadamard transform, a multi-qubit operation in quantum computing. It starts by introducing the idea of starting with a uniform superposition on all basis states, applying an oracle, and then applying a Hadamard transform to each qubit. The Hadamard transform is defined as a specific operation on single-qubit basis states, and its effect on a multi-qubit state is given by taking the product of the transformations on each individual qubit. The result of the superposition-oracle-Hadamard transform circuit is given by a messy equation, but it can be shown that the result of this operation does not depend on the secret lock combination, but rather on global properties of the state. The text concludes by discussing the limitations of using this circuit to determine the secret lock combination, as the outcomes are not orthogonal.

## Codercise A.5.1. 

Implement the circuit above, consisting of a Hadamard transform, an oracle, and a Hadamard transform. The oracle is provided as `oracle_matrix(combo)`, which you can invoke using the `QubitUnitary()` function.

Tip. To implement the Hadamard transform, apply PennyLane's `broadcast()` function instead of a for loop. This applies a unitary multiple times according to a given pattern, and for a specified set of wires.

In [10]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)


@qml.qnode(dev)
def hoh_circuit(combo):
    """A circuit which applies Hadamard-oracle-Hadamard and returns probabilities.

    Args:
    ------

        combo (list[int]): A list of bits representing a secret combination.

    Returns:
    --------

        list[float]: Measurement outcome probabilities.
    """

    qml.broadcast(qml.Hadamard, wires=list(range(n_bits)), pattern="single")
    qml.QubitUnitary(oracle_matrix(combo), wires=list(range(n_bits)))
    qml.broadcast(qml.Hadamard, wires=list(range(n_bits)), pattern="single")
    return qml.probs(wires=range(n_bits))

The circuit is designed to perform the Hadamard-Oracle-Hadamard (HOH) algorithm and returns the measurement outcome probabilities of a given secret combination.

The `hoh_circuit` function takes in an argument `combo`, a list of bits that represent a secret combination. The function then applies the Hadamard gate on all qubits using the `qml.broadcast` function, followed by the oracle operation represented by the `qml.QubitUnitary` function with the oracle matrix obtained from the `oracle_matrix` function and applied to all qubits. Finally, it applies another Hadamard gate on all qubits and returns the outcome probabilities using the `qml.probs` function.

The output of the circuit remains the same regardless of the secret combination, which makes it difficult to use the circuit to learn the secret combination directly. However, there may still be information that can be learned from the output of the circuit, similar to how the pair-testing algorithm applied to a lock with multiple solutions can still provide information.

# A.6 Deutsch-Jozsa

The Deutsch-Jozsa algorithm has a depth of 2 and makes a single oracle call to determine whether a function is constant or balanced. On the other hand, a deterministic classical algorithm requires exponential function calls to determine the same result, while a probabilistic classical algorithm requires a constant number of random function calls for a fixed error probability. The Deutsch-Jozsa algorithm offers a mild improvement over the probabilistic classical algorithm, providing the correct answer with probability 1.

## Codercise A.6.1.

<center>
<img src='./img/a.6.1.png' style='width:40%; height:40%;'>
</center>

Implement the circuit above for a set of solutions `combos`, and return probabilities. As before, you are given `multisol_oracle_matrix(combos)`, which returns the associated oracle in matrix form.

In [11]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)


@qml.qnode(dev)
def multisol_hoh_circuit(combos):
    """A circuit which applies Hadamard, multi-solution oracle, then Hadamard.

    Args:
    ------

        `combos` (`list[list[int]]`): A list of secret bit strings.

    Returns:
    --------

        `array[float]`: Probabilities for observing different outcomes.
    """

    qml.broadcast(qml.Hadamard, wires=list(range(n_bits)), pattern="single")
    qml.QubitUnitary(multisol_oracle_matrix(combos), wires=list(range(n_bits)))
    qml.broadcast(qml.Hadamard, wires=list(range(n_bits)), pattern="single")

    return qml.probs(wires=range(n_bits))

The function `multisol_hoh_circuit` implements a quantum circuit for the multi-solution Deutsch-Jozsa algorithm. The input to the function is a list of secret bit strings, represented as `combos`.

The circuit consists of three steps:

* Hadamard operation on each qubit, which prepares the initial state of the qubits in the superposition.
* Application of a multi-solution oracle `multisol_oracle_matrix`, which performs a unitary operation on the qubits. This operation is specified by the `combos` input and marks the solutions to the problem.
* Another application of Hadamard operation on each qubit, which takes the final state of the qubits back to the computational basis.

Finally, the function returns the probabilities of observing different outcomes when measuring the final state of the qubits, which are stored in an array.

The Deutsch-Jozsa algorithm is a quantum algorithm that can determine whether a given function (oracle) is either constant or balanced. The probability of observing the outcome "0" can be used to distinguish between these two cases.

When there are no solutions (all combinations are non-solutions), the probability of observing "0" is 1. This probability decreases as the number of solutions increases until it reaches 0 at the halfway point (half the combinations are solutions). From there, the probability climbs back up again and reaches 1 when all combinations are solutions.

The probability of observing "0" can be expressed as , where is the number of solutions and is the total number of combinations. If half the combinations are solutions, then . In this case, the function is considered balanced. If all or none of the combinations are solutions, then or , respectively, and the function is considered constant.

If the outcome "0" is measured, it indicates that the function is constant. If the outcome "0" is not measured, it indicates that the function is balanced.

## Codercise A.6.2. 

Implement the Deutsch-Jozsa algorithm. Given a constant or balanced function based on the `promise_var` flag, implement the circuit from above to determine which it is. The function `multisol_hoh_circuit` from the previous exercise is available for you to use.

In [12]:
def multisol_combo(n_bits, n_combs):
    """Generate n_combs random combinations of n_bits binary values.

    Args:
    ------

        `n_bits` (int): Number of bits in each combination.
        `n_combs` (int): Number of combinations.

    Returns:
    --------

        `list[list[int]]`: A list of combinations, where each combination is a list of binary values.
    """
    return [[random.choice([0, 1]) for i in range(n_bits)] for j in range(n_combs)]

In [13]:
def deutsch_jozsa(promise_var):
    """Implement the Deutsch-Jozsa algorithm and guess the promise variable.

    Args:
        promise_var (int): Indicates whether the function is balanced (0) or constant (1).

    Returns:
        int: A guess at the promise variable.
    """
    if promise_var == 0:
        how_many = 2 ** (n_bits - 1)
    else:
        how_many = np.random.choice([0, 2**n_bits])  # Choose all or nothing randomly
    combos = multisol_combo(n_bits, how_many)  # Generate random combinations

    # Use multisol function to run the Deutsch-Josza circuit, return the value corresponding to a balanced or constant function
    if np.isclose(multisol_hoh_circuit(combos)[0], 0):
        return 0
    else:
        return 1

This function implements the Deutsch-Jozsa algorithm. It first checks if the `promise_var` input is equal to 0 or 1, and sets the `how_many` variable accordingly. If `promise_var` is 0, then `how_many` is set to `2^(n_bits - 1)` (half of the possible combinations). If `promise_var` is 1, how_many is set to either 0 or `2^n_bits`, chosen randomly.

Next, the function generates the combos variable using the `multisol_combo` function, with the number of bits `n_bits` and the number of combinations `how_many`.

Finally, the function uses the `multisol_hoh_circuit` to run the Deutsch-Jozsa algorithm and returns 0 if the first element of the output is close to 0, indicating a balanced function, and 1 otherwise, indicating a constant function.